# Спринт  — Исследование и моделирование на данных NASDAQ

## Цель спринта
Провести полный цикл подготовки и первичного анализа данных NASDAQ: от визуального EDA в Power BI до построения базовых моделей машинного обучения для классификации движения цены (вверх/вниз) на следующий день.

---

## Этап 1 — Исследование в Power BI

### Задачи

#### 1. Подготовка данных
- Загрузить CSV-файл с историческими данными акций NASDAQ.
- Убедиться, что столбец `Date` распознан как дата (тип данных: дата).
- Удалить строки с пропущенными значениями.
- Создать отдельную таблицу с уникальными тикерами (`Symbol`), если нужно фильтровать по компаниям.

#### 2. Создание новых признаков
В редакторе Power Query или с помощью DAX создайте следующие поля:
- `Volatility = (High - Low) / Open` — относительная внутридневная волатильность.
- `Daily Return = (Close - Open) / Open` — доходность за день.
- `DayOfWeek = WEEKDAY(Date, 2)` — номер дня недели (1 = понедельник, 7 = воскресенье).
- `Day_Type`:
  ```DAX
  Day_Type = 
  SWITCH(TRUE(),
    [Close] > [Open], "Bull",
    [Close] < [Open], "Bear",
    "Neutral"
  )


#### 3. Визуализации

Создайте следующие визуализации в Power BI:

- **Тренд закрытия (Close) по дате**:
  - Используйте линейный график.
  - Добавьте срез (slicer) по тикеру (`Symbol`) для переключения между акциями.

- **Объём торгов (`Volume`) и скользящее среднее**:
  - Постройте линейный график `Volume` по дате.
  - Добавьте скользящую среднюю объёма (например, 7-дневную):
    - Через Power Query или DAX: `Volume_MA7 = AVERAGEX(LASTN(7, ...), Volume)`.

- **Волатильность**:
  - Постройте гистограмму признака `Volatility = (High - Low) / Open`.
  - Создайте столбчатую диаграмму средней волатильности по месяцам (используйте группировку по месяцу).

- **Доходность по дням недели**:
  - Используйте категорию `DayOfWeek`, созданную из даты.
  - Постройте диаграмму средних значений `Daily Return` по каждому дню недели.
  - При наличии custom visuals — добавьте boxplot по `Daily Return`.

- **Классификация дней (`Day_Type`)**:
  - Постройте круговую (pie chart) или столбчатую диаграмму распределения типов дней:
    - `"Bull"` — рост (`Close > Open`);
    - `"Bear"` — падение (`Close < Open`);
    - `"Neutral"` — почти без изменений.


#### 4. Интерактивный дашборд
Объедините визуализации на одном листе:

- Используйте срезы (slicers) для:
  - выбора тикера (`Symbol`);
  - выбора типа дня (`Day_Type`);
  - временного периода (по `Date`).
  
- Добавьте KPI-индикаторы:
  - Средняя доходность (среднее `Daily Return`);
  - Медианная волатильность;
  - Количество дней каждого типа (`Bull`, `Bear`, `Neutral`).

- Добавьте кнопку для экспорта отфильтрованных данных (опционально через Power BI Service).

#### 5. Выгрузка результата
- Отфильтруйте один тикер (например, `AAPL`), либо оставьте все.
- Убедитесь, что включены все необходимые поля:  
  `Date`, `Open`, `High`, `Low`, `Close`, `Volume`, `Volatility`, `Daily Return`, `DayOfWeek`, `Day_Type`.
- Сохраните отфильтрованный датасет в формате CSV для дальнейшей работы в Jupyter Notebook.

---

## Этап 2 — Моделирование в Jupyter Notebook

### Цель
Построить модели машинного обучения для классификации: вырастет ли цена акции на следующий день (0 или 1).

### Задачи

#### 1. Подготовка данных
- Загрузить CSV, экспортированный из Power BI.
- Преобразовать `Date` в формат datetime и отсортировать по дате.
- Добавить столбец `Target`:
  - `1`, если `Close_t+1 > Close_t`;
  - `0` — иначе.

#### 2. Feature Engineering
Создайте дополнительные признаки:
- Лаги: `Close_t-1`, `Volume_t-1`, `Daily Return_t-1`;
- Скользящие средние: `SMA_3`, `SMA_7`;
- Волатильность;
- Разница `Close - Open`;
- RSI, MACD, Bollinger Bands (опционально).

#### 3. Разделение данных
- Использовать `train_test_split` без перемешивания (`shuffle=False`);
- Масштабировать числовые признаки (`StandardScaler` или `MinMaxScaler`) для логистической регрессии.

#### 4. Обучение моделей
Обучить и сравнить следующие модели:
- `Logistic Regression`;
- `Random Forest`;
- `LightGBM`;
- `CatBoost`.

Каждую модель:
- Обучить на тренировочном наборе;
- Предсказать тестовые значения;
- Сохранить метрики: `Accuracy`, `F1`, `Precision`, `Recall`, `ROC-AUC`.

#### 5. Сравнение результатов
- Построить матрицы ошибок (confusion matrix) для всех моделей;
- Построить ROC-кривые;
- Сформировать таблицу сравнения метрик;
- Визуализировать важность признаков (где возможно).

#### 6. (Дополнительно) Бэктест стратегии
- Стратегия: покупать акцию, если модель предсказала рост (`Target = 1`);
- Рассчитать кумулятивную доходность по этой стратегии;
- Сравнить с “купи и держи” (Buy & Hold).

---

## Результаты спринта

- **Power BI**: файл с дашбордом, визуализациями и фильтрами;
- **CSV-файл**: подготовленные и очищенные данные;
- **Jupyter Notebook**: код моделей, оценки, визуализации;
- **Сравнительный анализ**: таблица результатов, графики, интерпретация;
- (Опционально) простая торговая симуляция на основе модели.


# Глоссарий терминов для проекта на данных NASDAQ

## 1. NASDAQ (National Association of Securities Dealers Automated Quotations)
- Вторая по капитализации биржа в мире (после NYSE).
- Электронная платформа, где торгуются в основном акции высокотехнологичных компаний.
- Примеры компаний: Apple (AAPL), Microsoft (MSFT), Amazon (AMZN), Tesla (TSLA), Meta (META).
- Индекс NASDAQ Composite включает тысячи акций этой биржи.

---

## 2. Основные рыночные показатели

### Open (Цена открытия)
Цена акции в начале торгового дня.

### Close (Цена закрытия)
Последняя цена акции на момент закрытия биржи.

### High / Low
- **High** — максимальная цена за день.
- **Low** — минимальная цена за день.

### Volume (Объём торгов)
Количество акций, которыми торговали за день. Используется для анализа ликвидности и интереса к бумаге.

### Volatility (Волатильность)
Измеряет колебания цены в течение дня.  
Формула: `(High - Low) / Open`.  
Высокая волатильность = больший риск и потенциал доходности.

### Daily Return (Дневная доходность)
Процентное изменение цены за день:  
`(Close - Open) / Open`.

### Target (Целевая переменная)
Класс, который модель предсказывает:  
`1`, если `Close(t+1) > Close(t)` — цена выросла;  
`0`, если упала или осталась прежней.

### Lag Features (Лаговые признаки)
Признаки, отражающие значения переменных в предыдущие дни:  
например, `Close_t-1`, `Volume_t-1`.

---

## 3. Технические индикаторы

### SMA (Simple Moving Average)
Простая скользящая средняя: усреднение цены за N дней.  
Помогает определить тренды.

### RSI (Relative Strength Index)
Индекс относительной силы.  
Показывает, перекуплен ли актив (RSI > 70) или перепродан (RSI < 30).

### MACD (Moving Average Convergence Divergence)
Разница между двумя скользящими средними.  
Используется для анализа тренда и его изменения.

### Bollinger Bands
Полосы вокруг скользящей средней на основе стандартного отклонения.  
Используются для оценки волатильности и возможных разворотов.

---

## 4. Визуализация и анализ

### Boxplot (Ящик с усами)
График, показывающий медиану, квартили и выбросы.  
Полезен для анализа распределения значений (например, доходности).

### Confusion Matrix (Матрица ошибок)
Матрица, показывающая количество верных и неверных предсказаний модели.

### ROC AUC (Receiver Operating Characteristic - Area Under Curve)
Метрика качества классификации.  
Показывает способность модели различать классы.  
1.0 — идеально, 0.5 — случайное угадывание.

---

## 5. Машинное обучение

### Feature Engineering
Создание новых признаков на основе исходных данных для улучшения качества модели.

### Train/Test Split
Разделение данных на обучающую и тестовую выборку.  
При временных рядах данные нельзя перемешивать.

### Scaling (Масштабирование признаков)
Приведение всех признаков к единому масштабу.  
Важно для моделей, чувствительных к величинам (например, Logistic Regression).

---

## 6. Модели

### Logistic Regression
Простая линейная модель классификации.  
Предсказывает вероятность принадлежности объекта к классу.

### Random Forest
Ансамбль из деревьев решений.  
Использует голосование по деревьям для получения предсказания.

### LightGBM
Градиентный бустинг с высокой скоростью и точностью.  
Подходит для больших и сложных датасетов.

### CatBoost
Градиентный бустинг от Яндекса.  
Эффективно работает с категориальными признаками и не требует сильной предобработки.

